# Logistic Regression

Logistic regression is a classic binary classification method that models the probability of an input belonging to the positive class by passing a linear combination through a sigmoid:

$$
h_{\mathbf{w},b}(\mathbf{x}) = \sigma\bigl(\mathbf{w}^\top \mathbf{x} + b\bigr),
\quad
\sigma(z) = \frac{1}{1 + e^{-z}}.
$$

## Prediction Step

1. Compute the linear score (logit):
   $$
   z = \mathbf{w}^\top \mathbf{x} + b.
   $$
2. Convert to probability:
   $$
   \hat y = \sigma(z).
   $$

## Loss Function

We use the binary cross‑entropy to measure how well the predicted probabilities match the true labels \(y^{(i)}\in\{0,1\}\):

$$
J(\mathbf{w}, b)
= -\frac{1}{m}\sum_{i=1}^m \Bigl[
y^{(i)}\ln(\hat y^{(i)}) \;+\;(1 - y^{(i)})\ln\bigl(1 - \hat y^{(i)}\bigr)
\Bigr].
$$

## Gradient Computation

The gradients of \(J\) with respect to the parameters are:

$$
\frac{\partial J}{\partial \mathbf{w}}
= \frac{1}{m}\sum_{i=1}^m \bigl(\hat y^{(i)} - y^{(i)}\bigr)\,\mathbf{x}^{(i)},
\qquad
\frac{\partial J}{\partial b}
= \frac{1}{m}\sum_{i=1}^m \bigl(\hat y^{(i)} - y^{(i)}\bigr).
$$

## Parameter Update (Gradient Descent)

With a learning rate \(\eta\), update parameters as:

$$
\mathbf{w} \leftarrow \mathbf{w} - \eta\,\frac{\partial J}{\partial \mathbf{w}},
\qquad
b \leftarrow b - \eta\,\frac{\partial J}{\partial b}.
$$

By iterating prediction → loss → gradient → update, the model converges to values of \(\mathbf{w}\) and \(b\) that best separate the two classes.


**Visualize the shape of the  Logistic function**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

from mlfs.logistic_regression import LogisticRegression as CustomLogisticRegression
from mlfs.preprocessing import train_test_split, standardize
from mlfs.metrics import (
    accuracy as custom_accuracy,
    precision as custom_precision,
    recall as custom_recall,
    f1_score as custom_f1,
)


In [1]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

z = np.linspace(-12, 12, 200)
fig = px.line(x=z, y=sigmoid(z),title='Logistic Function',template="plotly_dark")

fig.show()

In [2]:
df = pd.read_csv("../data/breast-cancer.csv")
df

In [4]:
fig = px.histogram(
    df,
    x='diagnosis',
    color='diagnosis', 
    template="plotly_dark"
)
fig.show()

In [4]:
fig = px.histogram(
    df,
    x='area_mean',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [5]:
fig = px.histogram(
    df,
    x='radius_mean',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [36]:
fig = px.histogram(
    df,
    x='perimeter_mean',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [37]:
fig = px.histogram(
    df,
    x='smoothness_mean',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [ ]:
fig = px.histogram(
    df,
    x='texture_mean',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [38]:
fig = px.scatter(
    df,
    x='symmetry_worst',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [39]:
fig = px.scatter(
    df,
    x='concavity_worst',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x='fractal_dimension_worst',
    color='diagnosis',
    template = 'plotly_dark')
fig.show()

<a id="4"></a>
<h1 style='background:#00EFFF;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '

# Data Preprocessing

In [3]:
df.drop('id', axis=1, inplace=True)

In [4]:
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int) 

In [5]:
corr = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr, cmap='mako_r',annot=True)
plt.show()

In [6]:
cor_target = abs(corr["diagnosis"])
relevant_features = cor_target[cor_target>0.2]
names = [index for index, value in relevant_features.items()]
names.remove('diagnosis')
X = df[names].values
y = df['diagnosis'].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train, mean, std = standardize(X_train, return_params=True)
X_test = (X_test - mean) / std

**When working with classification models, it's essential to evaluate how well they perform. Below are four key metrics commonly used to assess classification performance:**

---

## 1. Accuracy

**Formula:**

$$
\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}
$$

**Description:**
- Accuracy represents the proportion of correctly predicted samples among all predictions.
- It's a widely used and intuitive metric for evaluating model performance.

**Interpretation:**
- Higher accuracy generally indicates better overall performance.
- However, in cases of class imbalance, accuracy can be misleading.

---

## 2. Precision

**Formula:**

$$
\text{Precision} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Positives}}
$$

**Description:**
- Precision shows how many of the positive predictions were actually correct.
- It’s especially important when false positives are costly or problematic.

**Interpretation:**
- A high precision score means the model is rarely wrong when it predicts a positive.

---

## 3. Recall (Sensitivity)

**Formula:**

$$
\text{Recall} = \frac{\text{True Positives}}{\text{True Positives} + \text{False Negatives}}
$$

**Description:**
- Recall measures how many actual positive cases were correctly identified.
- It's crucial in situations where missing a positive instance is risky (e.g., in medical diagnoses).

**Interpretation:**
- High recall indicates the model successfully captures most positive cases.

---

## 4. F1-Score

**Formula:**

$$
\text{F1} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

**Description:**
- The F1-score is the harmonic mean of precision and recall.
- It provides a single score that balances both false positives and false negatives.

**Interpretation:**
- A high F1-score suggests that the model performs well in both identifying and correctly predicting positive instances.


<a id="4"></a>
<h1 style='background:#00EFFF;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '

# Comparing

In [9]:
custom_model = CustomLogisticRegression()
custom_model.fit(X_train, y_train, iterations= 1000,  plot_cost=False)

sk_model = SklearnLogisticRegression(max_iter=1000, solver='lbfgs')
sk_model.fit(X_train, y_train)

y_pred_custom = custom_model.predict(X_test)
y_pred_sk     = sk_model.predict(X_test)

acc_c  = custom_accuracy(y_test, y_pred_custom)
prec_c = custom_precision(y_test, y_pred_custom)
rec_c  = custom_recall(y_test, y_pred_custom)
f1_c   = custom_f1(y_test, y_pred_custom)

acc_s  = accuracy_score(y_test, y_pred_sk)
prec_s = precision_score(y_test, y_pred_sk)
rec_s  = recall_score(y_test, y_pred_sk)
f1_s   = f1_score(y_test, y_pred_sk)

df_results = pd.DataFrame({
    'model':     ['custom',       'sklearn'],
    'Accuracy':  [acc_c,          acc_s],
    'Precision': [prec_c,         prec_s],
    'Recall':    [rec_c,          rec_s],
    'F1-score':  [f1_c,           f1_s]
})

display(df_results)


In [11]:
def benchmark_logistic_custom_data(X, y, n_repeats=3, iterations=1000):
    """
    Compares the fit() and predict() times of a custom logistic regression implementation
    and sklearn's implementation on the given data X and y.

    Parameters:
        X (np.ndarray or pd.DataFrame): Input features.
        y (np.ndarray or pd.Series): Labels (0 or 1).
        n_repeats (int): Number of repetitions for averaging the timing.
        iterations (int): Number of training iterations for the custom model.

    Returns:
        pd.DataFrame: DataFrame containing fit and predict times.
    """
    custom_times = {'fit': [], 'predict': []}
    sklearn_times = {'fit': [], 'predict': []}

    for _ in range(n_repeats):
        # Custom model
        model = CustomLogisticRegression()
        t0 = time.perf_counter()
        model.fit(X, y, iterations=1000)
        t1 = time.perf_counter()
        model.predict(X)  
        t2 = time.perf_counter()
        custom_times['fit'].append(t1 - t0)
        custom_times['predict'].append(t2 - t1)

        # Sklearn model
        model = SklearnLogisticRegression(max_iter=1000, solver='lbfgs')
        t0 = time.perf_counter()
        model.fit(X, y)
        t1 = time.perf_counter()
        model.predict(X)  
        t2 = time.perf_counter()
        sklearn_times['fit'].append(t1 - t0)
        sklearn_times['predict'].append(t2 - t1)

    return pd.DataFrame({
        'custom_fit':      [np.mean(custom_times['fit'])],
        'custom_predict':  [np.mean(custom_times['predict'])],
        'sklearn_fit':     [np.mean(sklearn_times['fit'])],
        'sklearn_predict': [np.mean(sklearn_times['predict'])]
    })


In [12]:
df_times = benchmark_logistic_custom_data(X_train, y_train, n_repeats=5)
display(df_times)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].bar(['Custom', 'Sklearn'], [df_times['custom_fit'][0], df_times['sklearn_fit'][0]],
            color=['#1f77b4', '#ff7f0e'])
axes[0].set_title('Fit Time Comparison')
axes[0].set_ylabel('Time (seconds)')
axes[0].grid(axis='y', linestyle='--', alpha=0.7)

axes[1].bar(['Custom', 'Sklearn'], [df_times['custom_predict'][0], df_times['sklearn_predict'][0]],
            color=['#1f77b4', '#ff7f0e'])
axes[1].set_title('Predict Time Comparison')
axes[1].set_ylabel('Time (seconds)')
axes[1].grid(axis='y', linestyle='--', alpha=0.7)

plt.suptitle('Benchmark: Custom vs Sklearn Logistic Regression', fontsize=14)
plt.tight_layout()
plt.show()



In [ ]:
def benchmark_logistic(n_samples_list, n_features=10, n_repeats=3):
    """
    For each number of samples in `n_samples_list`:
      - generates binary classification data,
      - trains CustomLogisticRegression and measures the time for fit() and predict(),
      - trains Sklearn's LogisticRegression and measures the time for fit() and predict().
    Returns a DataFrame with columns:
      ['n_samples', 'custom_fit', 'custom_predict', 'sklearn_fit', 'sklearn_predict']
    """
    results = []
    for n in n_samples_list:
        X, y = make_classification(
            n_samples=n,
            n_features=n_features,
            n_informative=max(2, n_features // 2),
            n_redundant=0,
            random_state=42
        )
        
        custom_times = {'fit': [], 'predict': []}
        for _ in range(n_repeats):
            model = CustomLogisticRegression()
            t0 = time.perf_counter()
            model.fit(X, y, iterations = 1000)
            t1 = time.perf_counter()
            model.predict(X[:10])
            t2 = time.perf_counter()
            custom_times['fit'].append(t1 - t0)
            custom_times['predict'].append(t2 - t1)
        
        sklearn_times = {'fit': [], 'predict': []}
        for _ in range(n_repeats):
            model = SklearnLogisticRegression(max_iter=1000, solver='lbfgs')
            t0 = time.perf_counter()
            model.fit(X, y)
            t1 = time.perf_counter()
            model.predict(X[:10])
            t2 = time.perf_counter()
            sklearn_times['fit'].append(t1 - t0)
            sklearn_times['predict'].append(t2 - t1)
        
        results.append({
            'n_samples':       n,
            'custom_fit':      np.mean(custom_times['fit']),
            'custom_predict':  np.mean(custom_times['predict']),
            'sklearn_fit':     np.mean(sklearn_times['fit']),
            'sklearn_predict': np.mean(sklearn_times['predict']),
        })
        
    return pd.DataFrame(results)


In [15]:
def plot_benchmark(df, sample_col, custom_fit_col, sklearn_fit_col, custom_pred_col, sklearn_pred_col):
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(df[sample_col], df[custom_fit_col], label='Custom fit')
    plt.plot(df[sample_col], df[sklearn_fit_col], label='Sklearn fit')
    plt.xlabel('Number of samples')
    plt.ylabel('Fit time (s)')
    plt.title('Fit time comparison')
    plt.legend()
    plt.xscale('log')
    
    plt.subplot(1, 2, 2)
    plt.plot(df[sample_col], df[custom_pred_col], label='Custom predict')
    plt.plot(df[sample_col], df[sklearn_pred_col], label='Sklearn predict')
    plt.xlabel('Number of samples')
    plt.ylabel('Predict time (s)')
    plt.title('Predict time comparison')
    plt.legend()
    plt.xscale('log')
    
    plt.tight_layout()
    plt.show()


In [16]:
ns = [100, 1_000, 10_000, 100_000]
df_times = benchmark_logistic(ns, n_features=20, n_repeats=5)

plot_benchmark(df_times,
               sample_col='n_samples',
               custom_fit_col='custom_fit',
               sklearn_fit_col='sklearn_fit',
               custom_pred_col='custom_predict',
               sklearn_pred_col='sklearn_predict')

## Benchmark Results: Custom vs Sklearn Logistic Regression

The plots below compare the training and prediction times of a custom implementation of logistic regression with scikit-learn's `LogisticRegression`. The experiment was performed on datasets of increasing size (100, 1,000, 10,000, and 100,000 samples) using 20 features, with each measurement averaged over 5 runs.

### Fit Time Comparison (Left Plot)

- The custom model shows significantly higher training times compared to scikit-learn.
- This is expected due to the use of a plain **batch gradient descent** algorithm in the custom implementation, with a relatively small learning rate (`0.0001`) and potentially a large number of iterations.
- The model does include **early stopping based on gradient norm**, which helps avoid unnecessary iterations, but is still far less efficient than the **LBFGS solver** used by `sklearn`, which benefits from advanced optimization techniques and low-level performance tuning.
- As dataset size increases, the gap between the two implementations becomes larger.

### Predict Time Comparison (Right Plot)

- The custom model has a slightly faster prediction time, but the absolute difference is negligible (on the order of microseconds).
- This is because the custom `predict()` method uses a minimal implementation: just a dot product and a threshold (sigmoid + 0.5), with no input validation or type checks.
- Scikit-learn's model includes additional overhead to ensure robustness and compatibility, which slightly increases prediction time.

### Final Thoughts

- The custom implementation is ideal for **educational purposes** and understanding the mechanics of logistic regression.
- However, for large-scale or production tasks, `scikit-learn` remains the preferred option due to its **optimization, speed, and reliability**.
- Future improvements to the custom model could include adding **regularization**, **adaptive learning rates**, or using **Numba/Cython** to speed up computations.